In [2]:
# 0314 v2 - 가까운 두 개 마커로 위치 파악
# 마커 개수 늘림 시야에 3개까지 잘 잡히는데 가까운 2개를 고르려다 보니까 어떤 마커 2개를 고르는지에 따라서 좌표가 다르게 나타남
#v3: 여러개 중에 2개씩 이용해서 위치 뽑고 평균 내기
#v4: 칼만 필터  - v3보다 개선됨 (튀는 값이 줄어들었지만 여전히 많긴 함)

# 0317 v1: z축 거리 고정해서 입력하기 + 칼만필터까지 적용
# 정확도 높음 z값 -1.8m로 했는데 로봇에 부착하고 다시 거리 재보면 될듯
# v2: 마커 17번 빠졌던 거 추가
# v3: 마커 z축 회전 정렬 
#v4: 회전 정렬 결과 나타나게 opencv 3d 축 그리기 반영
# v5: 로봇 제어를 위한 yaw 행렬 추출  -- 아직 안됨 아래 코드 실행해보고 수정할 것 


# 0505 v1: 로봇 제어를 위한 yaw 행렬 추출 부터 ( 단, Euler 변환까지 적용)

## 성공 - 마커 90도씩 돌려보면 결과 잘 나옴 (천장 마커로는 아직 테스트 못함

# 0516 : 추출한 행렬가지고 로봇 제어 시도
# yaw 추출 
# 다중 waypoint 설정해서 움직임 제어



#12 -13-14-17-16-15-...  한 번 순환은 가능한데 계속 반복하게 만드는 부분 추가해야함

In [5]:
#!/usr/bin/env python
import rospy
import math
from geometry_msgs.msg import Twist
from geometry_msgs.msg import Pose2D

class ArucoPIDFollower:
    def __init__(self):
        rospy.init_node('aruco_pid_follower')
        self.cmd_pub = rospy.Publisher('/mobile_base/commands/velocity', Twist, queue_size=10)
        self.pose_sub = rospy.Subscriber('/robot_pose', Pose2D, self.pose_callback, queue_size=1)

        # 목표 경로 (x, y, yaw)
        self.waypoints = [
            (0.7, 0.7, 0.0),
            (2.3, 0.7, 0.0),
            (3.7, 0.7, 0.0),
            (3.7, 2.3, 0.0),
            (2.3, 2.3, 0.0),
            (0.7, 2.3, 0.0)
        ]
        self.current_pose = None
        self.current_index = 0

        # PID 제어 계수
        self.Kp_lin = 0.5
        self.Ki_lin = 0.0
        self.Kd_lin = 0.1
        self.Kp_ang = 1.0
        self.Ki_ang = 0.0
        self.Kd_ang = 0.2

        self.error_sum_lin = 0
        self.prev_error_lin = 0
        self.error_sum_ang = 0
        self.prev_error_ang = 0

        self.goal_tolerance = 0.15  # 도착 허용 거리

        self.rate = rospy.Rate(10)  # 10Hz
        self.run()

    def pose_callback(self, msg):
        self.current_pose = msg

    def compute_pid(self, error, prev_error, error_sum, Kp, Ki, Kd, limit):
        error_sum += error
        d_error = error - prev_error
        output = Kp * error + Ki * error_sum + Kd * d_error
        output = max(min(output, limit), -limit)  # 속도 제한
        return output, error_sum, error

    def normalize_angle(self, angle):
        while angle > math.pi:
            angle -= 2.0 * math.pi
        while angle < -math.pi:
            angle += 2.0 * math.pi
        return angle

    def run(self):
        while not rospy.is_shutdown():
            if self.current_pose is None or self.current_index >= len(self.waypoints):
                self.rate.sleep()
                continue

            target_x, target_y, _ = self.waypoints[self.current_index]
            dx = target_x - self.current_pose.x
            dy = target_y - self.current_pose.y
            distance = math.hypot(dx, dy)

            # 방향 오차 계산
            desired_yaw = math.atan2(dy, dx)
            yaw_error = self.normalize_angle(desired_yaw - self.current_pose.theta)

            # PID 계산
            lin_vel, self.error_sum_lin, self.prev_error_lin = self.compute_pid(
                distance, self.prev_error_lin, self.error_sum_lin, self.Kp_lin, self.Ki_lin, self.Kd_lin, 0.3
            )
            ang_vel, self.error_sum_ang, self.prev_error_ang = self.compute_pid(
                yaw_error, self.prev_error_ang, self.error_sum_ang, self.Kp_ang, self.Ki_ang, self.Kd_ang, 1.0
            )

            cmd = Twist()
            if distance > self.goal_tolerance:
                cmd.linear.x = lin_vel
                cmd.angular.z = ang_vel
            else:
                rospy.loginfo(f"🟩 Waypoint {self.current_index} reached")
                self.current_index += 1

            self.cmd_pub.publish(cmd)
            self.rate.sleep()

        # 정지 명령
        self.cmd_pub.publish(Twist())
        rospy.loginfo("✅ Path following complete.")

if __name__ == '__main__':
    a = ArucoPIDFollower()
    a.run()


C:\Users\user\AppData\Local\Temp\ipykernel_8004\725015293.py:136: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  cx = float(cx)
C:\Users\user\AppData\Local\Temp\ipykernel_8004\725015293.py:137: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  cy = float(cy)
C:\Users\user\AppData\Local\Temp\ipykernel_8004\725015293.py:138: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  cz = float(cz)


[INFO] Position: (0.00, 0.00, 0.00), Yaw: -1.56°
[INFO] Position: (0.03, 0.00, -0.02), Yaw: -1.56°
[INFO] Position: (0.14, -0.00, -0.09), Yaw: -0.77°
[INFO] Position: (0.45, -0.00, -0.28), Yaw: 0.21°
[INFO] Position: (1.04, -0.01, -0.63), Yaw: 0.21°
[INFO] Position: (1.80, -0.02, -1.11), Yaw: 2.66°
[INFO] Position: (2.56, -0.02, -1.57), Yaw: 2.66°
[INFO] Position: (3.14, -0.02, -1.93), Yaw: 3.27°
[INFO] Position: (3.50, -0.02, -2.16), Yaw: 3.27°
[INFO] Position: (3.69, -0.02, -2.28), Yaw: 3.86°
[INFO] Position: (3.74, -0.02, -2.31), Yaw: 3.86°
[INFO] Position: (3.72, -0.01, -2.30), Yaw: 3.42°
[INFO] Position: (3.64, -0.01, -2.25), Yaw: 3.42°
[INFO] Position: (3.32, -0.01, -2.04), Yaw: -3.68°
[INFO] Position: (3.21, -0.00, -1.97), Yaw: -3.68°
[INFO] Position: (3.33, 0.00, -2.04), Yaw: 2.69°
[INFO] Position: (3.23, 0.01, -1.97), Yaw: 2.12°
[INFO] Position: (3.15, 0.02, -1.92), Yaw: 2.12°
[INFO] Position: (3.09, 0.02, -1.88), Yaw: 1.78°
[INFO] Position: (3.03, 0.02, -1.84), Yaw: 1.78°
[IN